In [1]:
import json
import operator
import Stemmer

from sklearn.model_selection import StratifiedKFold
import numpy as np

In [2]:
f = open('kzn-dsc/dataset.json')
jobs = json.load(f)
print(len(jobs))
jobs = [job for job in jobs if job['text'] != 'This message was deleted.']
print(len(jobs))

954
954


In [3]:
from collections import defaultdict

In [4]:
counter = defaultdict(int)

for job in jobs:
    for reaction in job['reactions']:
        # print(reaction)
        counter[reaction] += job['reactions'][reaction]

In [5]:
sorted_reactions = sorted(counter.items(), key=operator.itemgetter(1))

In [6]:
sorted_reactions[-30:]

[('ico', 226),
 ('grammar', 234),
 ('venheads', 238),
 ('money_mouth_face', 249),
 ('heavy_plus_sign', 249),
 ('tinkoff', 269),
 ('rowing-galera', 279),
 ('+1::skin-tone-6', 308),
 ('nor', 340),
 ('mickey', 347),
 ('ramen', 374),
 ('hankey', 377),
 ('wat', 391),
 ('chains', 403),
 ('joy', 408),
 ('noexcel', 426),
 ('putin', 430),
 ('moneys', 442),
 ('facepalm', 553),
 ('+1::skin-tone-2', 622),
 ('moneybag', 663),
 ('sberbank', 770),
 ('eww', 881),
 ('fireball', 917),
 ('notbad', 1694),
 ('fire', 2089),
 ('galera', 3727),
 ('+1', 5590),
 ('ban', 6281),
 ('fork', 7084)]

In [48]:
good_reactions = ['+1', 'fire', 'notbad', 'fireball', 'moneybag', 'heavy_plus_sign', 'money_mouth_face', '+1::skin-tone-2']
bad_reactions = ['ban', 'fork', 'galera', 'eww', 'facepalm', 'wat', 'rowing-galera', 'are_you_fucking_kidding_me', 'noexcel']

In [49]:
amount_of_good_reactions = sum([counter[reaction] for reaction in good_reactions])
amount_of_bad_reactions = sum([counter[reaction] for reaction in bad_reactions])

In [50]:
print(amount_of_good_reactions)
print(amount_of_bad_reactions)

12073
19831


In [10]:
stemmer = Stemmer.Stemmer('russian')

In [10]:
from nltk.stem.snowball import EnglishStemmer, RussianStemmer

ru_stemmer = RussianStemmer()
eng_stemmer = EnglishStemmer()

In [11]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
import re

def perfotm_transformation(text):
    text = text.lower()
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)
    words_without_punctuation = text.split()
    # 3. Stem words
    stemmed_words = [eng_stemmer.stem(ru_stemmer.stem(word)) for word in words_without_punctuation]
    
    return ' '.join(stemmed_words)

In [13]:
jobs[4]['text']

'Привет! Мы в Dbrain (<https://dbrain.io>) снова (вот в прошлый раз: <https://opendatascience.slack.com/archives/C04DA5FUF/p1519145569000376>) ищем дата-саентистов, на этот раз нам нужно два человека: - синьор - джуниор плюс (знает основы, уверенно идет к миддлу) Требования: - Опыт работы с современным DL фреймворкам (pytorch, tf, keras, mxnet). Мы в основном предпочитаем pytorch, но это не принципиально; - Опыт работы с python (scipy stack); - Опыт работы с дженерик моделями sklearn, xgboost, etc; - Опыт работы в ОС Linux (GNU tools, bash); - Опыт запуска моделей в лайт прод, показ MVP. Будет плюсом:: - Знает SQL; - Знание C/C++. При наличии убедительных аргументов полная свобода в принятии ключевых решений в выборе подходов / алгоритмов / фреймворков. Ну и мы всегда готовы советовать и помогать со своей стороны. Перспективы: - Стать лидом отдельной команды, разрабатывающей конкретный проект; - Возможность реализовать свою амбициозную идею. Условия: - Красивый офис между Новокузнецкой

In [14]:
perfotm_transformation(jobs[4]['text'])

'привет мы в dbrain https dbrain i снов вот в прошл раз https opendatascienc slack com archiv c04da5fuf p1519145569000376 ищ дат саентист на этот раз нам нужн два человек синьор джуниор плюс знает основ уверен идет к миддл требован оп работ с современ dl фреймворк pytorch tf kera mxnet мы в основн предпочита pytorch но эт не принципиальн оп работ с python scip stack оп работ с дженерик модел sklearn xgboost etc оп работ в ос linux gnu tool bash оп запуск модел в лайт прод показ mvp будет плюс знает sql знан c c при налич убедительн аргумент полн свобод в принят ключев решен в выбор подход алгоритм фреймворк ну и мы всегд готов советова и помога со сво сторон перспектив стат лид отдельн команд разрабатыва конкретн проект возможн реализова сво амбициозн ид услов красив офис межд новокузнецк и павелецк на кухн ест печен ча кофемашин и все необходим для сборк бутерброд оформлен по тк джуниор до 120к синьор до 250к на рук резюм лучш всег присыла на mailt job dbrain i job dbrain i в тем лучш

In [15]:
transformed_texts = [perfotm_transformation(job['text']) for job in jobs]

In [16]:
def is_job_galera(job):
    positive_reactions = sum([job['reactions'][reaction] for reaction in job['reactions'] if reaction in good_reactions])
    negative_reactions = sum([job['reactions'][reaction] for reaction in job['reactions'] if reaction in bad_reactions])
    return negative_reactions > positive_reactions

In [51]:
y_labels = []
for job in jobs:
    if is_job_galera(job):
        y_labels.append(1)
    else:
        y_labels.append(0)

In [52]:
print(sum(y_labels))

404


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(transformed_texts)
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

In [54]:
X_tfidf

<954x14421 sparse matrix of type '<class 'numpy.float64'>'
	with 133577 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
logit = LogisticRegression(C=1, random_state=19)
logit.fit(X_tfidf, y_labels)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=19, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
from sklearn.metrics import accuracy_score
predictions = logit.predict(X_tfidf)
print(accuracy_score(predictions, y_labels))

0.8825995807127882


In [58]:
sum(predictions)

306

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer_word = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 5))
train_word = vectorizer_word.fit_transform(transformed_texts)

In [60]:
logit = LogisticRegression(C=1, random_state=19)
logit.fit(train_word, y_labels)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=19, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
predictions = logit.predict(train_word)
print(accuracy_score(predictions, y_labels))

0.9769392033542977


In [65]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=15)
clf = clf.fit(train_word, y_labels)

In [66]:
predictions = clf.predict(train_word)
print(accuracy_score(predictions, y_labels))

0.8522012578616353


In [67]:
from sklearn.tree import export_graphviz
export_graphviz(clf, out_file='small_tree_1.dot', filled=True, feature_names=vectorizer_word.get_feature_names())
# для этого понадобится библиотека pydot (pip install pydot)
!dot -Tpng 'small_tree.dot' -o 'small_tree_1.png'

In [29]:
job_index = 850
print(jobs[job_index])
print(y_labels[job_index])
print(transformed_texts[job_index])

{'text': 'Требуется Senior Data scientist в международную компанию. OSA Hybrid Platform – сервис на основе BIG DATA платформы, управляемый прикладным Искусственным Интеллектом, повышающий уровень наличия товара на полке. Сервис работает в режиме реального времени. В проекте участвуют торговые сети, входящие в ТОП 10 ритейлеров: Перекресток, Dixy, Auchan, METRO Cash &amp; Carry, АВ Daily и пр. Так же, задействованы крупнейшие компании производители, среди них: Pepsi, Coca-Cola, Danone, JTI, Mars, Loreal, SanInBev, Efes, Unilever, Эфко и пр. Обязанности: • Менторство DS команды проекта. • Расширение круга гипотез в процессе анализа поведения потребителей в офф-лайн ритейле (фуд). • Расширение круга факторов, доступных для анализа (meta data, open data). • Проектирование и реализация алгоритмов прогноза покупательского поведения основанные на больших данных, используя машинное обучение. • Дата инжиниринг. • Покупательская сегментация и целевой маркетинг. • Курирование работы по созданию К

In [68]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(predictions, y_labels))
print(recall_score(predictions, y_labels))

0.9282178217821783
0.7700205338809035


In [102]:
def fit_predict(model, X, y):
    model.fit(X, y)
    predictions = model.predict(X)
    print("---------Training--------")
    print("Accuracy score: " + str(accuracy_score(y, np.where(predictions > 0.5, 1, 0))))
    print("Precision score: " + str(precision_score(y, np.where(predictions > 0.5, 1, 0))))
    print("Recall score: " + str(recall_score(y, np.where(predictions > 0.5, 1, 0))))

In [69]:
fit_predict(clf, X_tfidf, y_labels)

---------Training--------
Accuracy score: 0.7987421383647799
Precision score: 0.6840277777777778
Recall score: 0.9752475247524752


In [33]:
accuracy_score(clf.predict(X_tfidf), y_labels)

0.6331236897274634

In [34]:
fit_predict(logit, train_word, y_labels)

---------Training--------
Accuracy score: 0.9748427672955975
Precision score: 1.0
Recall score: 0.9392405063291139


In [160]:
from sklearn.model_selection import KFold

def run_cv(model, X, y, test):
    n_folds = 10
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
    #skf = KFold(n_splits=10, random_state=0)
    data_x = X
    data_y = y
    pred = []
    models = []
    tfidfs = []
    for train_index, test_index in skf.split(data_x, data_y):
        
        model = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.025,
                        n_estimators=20)
        
        x_train, x_test = np.array(data_x)[train_index], np.array(data_x)[test_index]
        y_train, y_test = np.array(data_y)[train_index], np.array(data_y)[test_index]
        
        tfidf = TfidfVectorizer(sublinear_tf=True, analyzer='word', stop_words='english', ngram_range=(1, 1))
        #tfidf = vectorizer_word
        tfidf.fit(x_train)
        
        x_train = tfidf.transform(x_train)
        x_test = tfidf.transform(x_test)
        test_tfidf = tfidf.transform(test)
        
#         char_vectorizer = TfidfVectorizer(
#                             sublinear_tf=True,
#                             analyzer='char',
#                             stop_words='english',
#                             ngram_range=(2, 6),
#                             max_features=50000)
#         char_vectorizer.fit(X)
        
#         train_char_features = char_vectorizer.transform(x_train)
#         test_char_features = char_vectorizer.transform(x_test)
#         tfidf = vectorizer_word
#         x_train = tfidf.transform(x_train)
#         x_test = tfidf.transform(x_test)
#         x_train = hstack([train_char_features, x_train])
#         x_test = hstack([test_char_features, x_test])
        fit_predict(model, x_train, y_train)
        y_pred = model.predict(x_test)
        # score = accuracy_score(y_test, y_pred)
        prediction = model.predict(test_tfidf)
        pred.append(prediction[:])
        print(sum(y_pred))
        print("---------Validation------")
        print("Accuracy score: " + str(accuracy_score(y_test, np.where(y_pred > 0.5, 1, 0))))
        print("Precision score: " + str(precision_score(y_test, np.where(y_pred > 0.5, 1, 0))))
        print("Recall score: " + str(recall_score(y_test, np.where(y_pred > 0.5, 1, 0))))
        models.append(model)
        tfidfs.append(tfidf)
    return pred, models, tfidfs

In [141]:
from sklearn.model_selection import train_test_split, KFold

train, test, y_train, y_test = train_test_split(transformed_texts, y_labels, test_size = 0.2, random_state = 42)

In [150]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.025,
                        n_estimators=20)

In [155]:
y_pred = run_cv(gbm, train, y_train, test)

y_pred = np.mean(y_pred, axis = 0)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("---------Hold-out---------")
print("Accuracy score: " + str(accuracy_score(y_test, y_pred)))
print("Precision score: " + str(precision_score(y_test, y_pred)))
print("Recall score: " + str(recall_score(y_test, y_pred)))

---------Training--------
Accuracy score: 0.8493894165535957
Precision score: 0.9952153110047847
Recall score: 0.6540880503144654
10.760597933680637
---------Validation------
Accuracy score: 0.6538461538461539
Precision score: 1.0
Recall score: 0.18181818181818182
---------Training--------
Accuracy score: 0.8439620081411127
Precision score: 0.9765258215962441
Recall score: 0.6540880503144654
11.058846183582684
---------Validation------
Accuracy score: 0.6153846153846154
Precision score: 1.0
Recall score: 0.09090909090909091
---------Training--------
Accuracy score: 0.8466757123473542
Precision score: 0.9812206572769953
Recall score: 0.6572327044025157
11.576782289081711
---------Validation------
Accuracy score: 0.5
Precision score: 0.3333333333333333
Recall score: 0.18181818181818182
---------Training--------
Accuracy score: 0.8263229308005428
Precision score: 0.9896907216494846
Recall score: 0.6037735849056604
11.55725568051095
---------Validation------
Accuracy score: 0.6153846153846

In [143]:
sum(y_test)

75

In [144]:
sum(y_pred)

32

In [145]:
y_test - y_pred

array([ 0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  1,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  1, -1,  0,  1,  0,  0,  0,  1,  1,  1,
        1,  0,  0,  1,  0,  1,  1,  1,  0,  1,  1,  0,  0,  1,  0, -1,  0,
        0,  1,  0,  1,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,
        1,  0,  0,  1,  0,  0,  0,  1,  1, -1,  0,  0,  0,  0,  1,  0,  1,
        0, -1,  1,  1,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  1,
       -1,  0,  1,  0,  1,  0,  0,  1,  1,  1,  0,  0,  0,  0,  1, -1,  0,
       -1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,
        0,  1, -1,  0,  1,  1, -1, -1,  0, -1,  1,  0,  0,  1,  1,  1,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,
       -1,  1,  0,  0])

In [156]:
train[0]

'дел в том что формальн ваканс соответств требован оформлен описан компан ест хот и ироничн написа ваканс четк указа треб дополнительн человек в поездк студент стажер и продавец оплат указа есл вы дума что мне за не стыдн вы ошиба то что он довольн развязн язык написа я соглас эт как бы парод на всяк лев маркетингов агентств котор мы конечн сам и явля и понима эт но хот у нас маркетингов агентств никт не люб имен так компан врод наш дела больш и очен тяжел работ по привлечен иностра денег в отечествен команд я горж что к эт причаст да конкуренц огромн и огромн част проект не продвига дальш пресейл о чем честн и написа в ваканс задач датамонстр так выход на заказчик получа у них четк постановк задач по форм передава эт форм отечественнвм команд и получа от команд предложен для заказчик тож качествен и быстр оформлен вот эт взаимодейств межд иностра заказчик и отечествен команд част явля сам узк мест част техническ российск команд отличн подкова но из за тог что он например плох знает ан

In [177]:
def process_text(text):
    formatted_text = perfotm_transformation(text)
    prob = []
    for model, tfidf in zip(models, tfidfs):
        features = tfidf.transform([formatted_text])
        prob.append(model.predict(features))
        
    return (np.mean(prob) > 0.5)

In [161]:
y_pred, models, tfidfs = run_cv(gbm, transformed_texts, y_labels, test)

y_pred = np.mean(y_pred, axis = 0)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("---------Hold-out---------")
print("Accuracy score: " + str(accuracy_score(y_test, y_pred)))
print("Precision score: " + str(precision_score(y_test, y_pred)))
print("Recall score: " + str(recall_score(y_test, y_pred)))

---------Training--------
Accuracy score: 0.8100233100233101
Precision score: 0.9854368932038835
Recall score: 0.559228650137741
40.94015226262435
---------Validation------
Accuracy score: 0.5833333333333334
Precision score: 0.5384615384615384
Recall score: 0.17073170731707318
---------Training--------
Accuracy score: 0.8193473193473193
Precision score: 0.9905660377358491
Recall score: 0.5785123966942148
40.50830636933579
---------Validation------
Accuracy score: 0.5625
Precision score: 0.46153846153846156
Recall score: 0.14634146341463414
---------Training--------
Accuracy score: 0.8146853146853147
Precision score: 0.9722222222222222
Recall score: 0.5785123966942148
41.262836761423294
---------Validation------
Accuracy score: 0.6041666666666666
Precision score: 0.6
Recall score: 0.21951219512195122
---------Training--------
Accuracy score: 0.8216783216783217
Precision score: 0.9952830188679245
Recall score: 0.581267217630854
40.958168342239425
---------Validation------
Accuracy score:

In [162]:
models

[LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.025, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.025, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.025, max_depth=-1,
  

In [185]:
index = 801
print(transformed_texts[index])
print(y_labels[index])
print(process_text(jobs[index]['text']))

треб middl senior te lead dat engin в компан ciklum с локац ки ил львов на ваш усмотрен flag u обязан • выбор и интеграц фреймворк big dat необходим для решен бизнес задач клиент • возможн участ в пре сейл общен с клиент • реализац etl • разработк backend на питон • определен политик хранен дан требован • solid understand of distribut comput principl • manag of hadoop cluster with all includ servic and configur of associ hardwar and softwar • python knowledg might be scal • proficienc with hadoop or cassandr map reduc hdfs • experienc with build stre process system use solut such as storm or spark stream • good knowledg of big dat queri tool such as pig hiv and impa • experienc with spark • experienc with integr of dat from multipl dat sourc • experienc with nosql databas such as hbas • experienc with рдбмс'с lik oracl and mysql • knowledg of various etl techniqu and framework • experienc with various messag system such as kafk experienc with big dat ml toolkit such as mahout sparkml o

In [169]:
jobs[0]['text']

'Ребят, привет! заранее извиняюсь, если пишу не в нужном формате (старался по возможности придерживаться), один хороший знакомый просил найти людей на проекты, связанные с анализом данных (все задачи и клиенты, описанные ниже проверены мной на адекватность): - Рекомендательная система для сети фитнес-клубов (данных много, есть полное их описание, семплы - гавернанс в компании на высшем уровне) - Рекомендательная система для крупной авиакомпании (тут задача в первую очередь - общаться с заказчиком и свести задачу к формальному описанию/ТЗ) По первому проекту - уже есть четкое понимание самой задачи, pipeline, все механики и превентивные меры, есть семплы данных, а также примерные требуетмые метрики качества. По второму - требуется общаться с заказчиком (довольно продвинутым) для более детальной проработки кейса Требования к кандидатам (очень субьективные, поэтому все равно пишите): наличие завершенных и внедренных аналитических продуктов. Тут не требуется выжимать доли качества, а важна

In [186]:
import pickle

In [187]:
pickle.dump( models, open( "models.p", "wb" ) )
pickle.dump( tfidfs, open( "tfidfs.p", "wb" ) )

In [188]:
def echo(bot, update):
    """Echo the user message."""
    # update.message.reply_text(update.message.text)
    if process_text(update.message.text):
        update.message.reply_text('👎')
    else:
        update.message.reply_text('👍')
    # update.message.reply_text('👎👍')
    

In [189]:
jobs[801]

{'text': 'Требуется *Middle/Senior/Team Lead Data Engineer* в компанию *Ciklum* с локацией *Киев* или *Львов* (на ваше усмотрение) :flag-ua: *Обязаности:* • выбор и интеграция фреймворков big data, необходимых для решения бизнес задач клиентов • возможное участие в пре-сейлах, общение с клиентами • реализация ETL • разработка backend на питоне • определении политики хранения данных *Требования:* • solid understanding of distributed computing principles • Management of Hadoop cluster, with all included services and configuration of associated hardware and software • Python knowledge, might be Scala • Proficiency with Hadoop or Cassandra, Map Reduce, HDFS • Experience with building stream-processing systems, using solutions such as Storm or Spark-Streaming • Good knowledge of Big Data querying tools, such as Pig, Hive, and Impala • Experience with Spark • Experience with integration of data from multiple data sources • Experience with NoSQL databases, such as HBase • Experience with RDBM

In [1]:
test

NameError: name 'test' is not defined